# Benchmarking distributed training with keras

Adapted from tensorflow distributed tutorial at https://www.tensorflow.org/tutorials/distribute/keras

## Overview

This notebook allows to you measure the first and second epoch train time for training deep learning models on multiple GPUs using the tensorflow MirroredStrategy for keras API. 

This experiment varies the following:

* Dataset size: MNIST digits repeated 1x, 4x, 8x = 60k, 240k, 480k training images
* Model size: Small with 402k trainable parameters, large with 2.6m trainable parameters. Both: adam optimizer, cross entropy loss
* Batch size: 128, 256, 512 images
* GPUs: I used: GCP n1-highmem-2 (2 vCPUs, 13 GB memory) with {1, 2, 4} NVIDIA Tesla K80 GPUs

And then it records:

* First epoch train time: incurs any startup costs
* Second epoch train time: representative of future epoch train times since training incurs same number of operations/epoch


### Import dependencies

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import time
import os
import json

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4657818615328315539
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 35951343315859641
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4858784739591713473
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16807915064472844064
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11269973607
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 4274797351884350489
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
, name: "/device:GPU:1"
device_type: "GPU"
memory_lim

In [3]:
print(tf.__version__)

2.0.0


## Helper functions

In [4]:
def get_dataset(N, dataset='mnist'):
    '''
    Download the MNIST dataset and load it from [TensorFlow Datasets]
    (https://www.tensorflow.org/datasets). This returns a dataset in `tf.data` format. 
    The helper function `get_dataset` also handles repeating the dataset N times 
    when we want to scale it up for the experiment.
    '''
    d = tfds.load(name=dataset, as_supervised=True)
    mnist_train, mnist_test = d['train'], d['test']

    for i in range(1, N):
        d = tfds.load(name=dataset, as_supervised=True)
        single_mnist_train, single_mnist_test = d['train'], d['test']

        mnist_train = mnist_train.concatenate(single_mnist_train)
        mnist_test = mnist_test.concatenate(single_mnist_test)

    return mnist_train, mnist_test

def scale(image, label):
    '''
    Pixel values, which are 0-255, [have to be normalized to the 0-1 range]
    (https://en.wikipedia.org/wiki/Feature_scaling). Define this scale in a function.
    '''
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

def define_small_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
    return model

def define_large_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(128, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(256, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(512, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])

    model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
    return model

def decay(epoch):
    '''
    Function for decaying the learning rate.
    You can define any decay function you need.
    '''
    if epoch < 3:
        return 1e-3
    elif epoch >= 3 and epoch < 7:
        return 1e-4
    else:
        return 1e-5

## Define experiment
The experiment function takes in the three variables we are interested in: number of times to repeat the MNIST dataset, batch_size (per replica), and the number of GPUs to use. It will save the time it takes to train the first and second epochs.

In [8]:
results = []


def experiment(define_model_fnc, n_dataset_repeat, batch_size_per_replica, n_gpus, record_results=False):    
    
    # define data parallel strategy if using more than one gpu
    if n_gpus > 1:
        strategy = tf.distribute.MirroredStrategy(devices=["/gpu:{}".format(i) for i in range(n_gpus)])
        print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

    # set up batch size var. depends on how many gpus are being used
    BUFFER_SIZE = 10000

    if n_gpus > 1:
        BATCH_SIZE = batch_size_per_replica * strategy.num_replicas_in_sync
    else:
        BATCH_SIZE = batch_size_per_replica
    
    # download and process dataset
    mnist_train, mnist_test = get_dataset(n_dataset_repeat, 'mnist')
    train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)
    
    # Create and compile the Keras model in the context of `strategy.scope`.
    if n_gpus>1:
        with strategy.scope():
            model = define_model_fnc()
    else:
        model = define_model_fnc()
    
    # define callback that will record the epoch train time
    epoch_times = []
    class timecallback(tf.keras.callbacks.Callback):            
        def on_epoch_begin(self,epoch,logs={}):
            self.starttime = time.clock()
            
        def on_epoch_end(self,epoch,logs = {}):
            epoch_times.append(time.clock() - self.starttime)

    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir='./logs'),
        tf.keras.callbacks.LearningRateScheduler(decay),
        timecallback(),
    ]
    
    model.fit(train_dataset, epochs=2, callbacks=callbacks)
    
    if record_results:
        with open('tensorflow_results.txt', 'a') as f:
            f.write(json.dumps(
                        {'n_dataset_repeat': n_dataset_repeat,
                        'batch_size': batch_size_per_replica,
                        'n_gpus': n_gpus,
                        'first epoch time': epoch_times[0],
                        'second epoch time': epoch_times[1]}) + '\n')


In [9]:
MODELS = [define_small_model, define_large_model]
DATASET_REPEATS = [1,4,8]
BATCH_SIZES = [128, 256, 512]
GPU_NUMS = [i for i in range(len(tf.config.experimental.list_physical_devices('GPU'))+1) if i in (1,2,4,8)]

for d in DATASET_REPEATS:
    for b in BATCH_SIZES:
        for g in GPU_NUMS:
            for m in MODELS:
                print('\n' + '*'*80 + '\n')
                print('Now training: {} on dataset repeated {}x with batch size {} on {} gpu(s)'.format(m.__name__, d, b, g))
                experiment(m,d,b,g, record_results=True)


********************************************************************************

Now training: define_small_model on dataset repeated 1x with batch size 128 on 1 gpu(s)
Epoch 1/2
469/469 [==============================] - 19s 42ms/step - loss: 0.2707 - accuracy: 0.9246
Epoch 2/2
469/469 [==============================] - 3s 7ms/step - loss: 0.0840 - accuracy: 0.9757

********************************************************************************

Now training: define_large_model on dataset repeated 1x with batch size 128 on 1 gpu(s)
Epoch 1/2
469/469 [==============================] - 32s 68ms/step - loss: 0.2091 - accuracy: 0.9338
Epoch 2/2
469/469 [==============================] - 15s 32ms/step - loss: 0.0554 - accuracy: 0.9834

********************************************************************************

Now training: define_small_model on dataset repeated 1x with batch size 128 on 2 gpu(s)
Number of devices: 2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Epoch 1/2
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


...

1875/1875 [==============================] - 153s 82ms/step - loss: 0.0925 - accuracy: 0.9733
Epoch 2/2
1875/1875 [==============================] - 22s 11ms/step - loss: 0.0134 - accuracy: 0.9964

********************************************************************************

Now training: define_large_model on dataset repeated 8x with batch size 128 on 2 gpu(s)
Number of devices: 2
Epoch 1/2
      1/Unknown - 7s 7s/step - loss: 2.3044 - accuracy: 0.0703WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.124822). Check your callbacks.


1875/1875 [==============================] - 178s 95ms/step - loss: 0.0600 - accuracy: 0.9811
Epoch 2/2
1875/1875 [==============================] - 72s 38ms/step - loss: 0.0087 - accuracy: 0.9972

********************************************************************************

Now training: define_small_model on dataset repeated 8x with batch size 256 on 1 gpu(s)
Epoch 1/2
1875/1875 [==============================] - 151s 81ms/step - loss: 0.0865 - accuracy: 0.9754
Epoch 2/2
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0123 - accuracy: 0.9967

********************************************************************************

Now training: define_large_model on dataset repeated 8x with batch size 256 on 1 gpu(s)
Epoch 1/2
1875/1875 [==============================] - 216s 115ms/step - loss: 0.0601 - accuracy: 0.9809
Epoch 2/2
1875/1875 [==============================] - 103s 55ms/step - loss: 0.0088 - accuracy: 0.9972

*********************************************

938/938 [==============================] - 153s 163ms/step - loss: 0.1229 - accuracy: 0.9657
Epoch 2/2
938/938 [==============================] - 14s 15ms/step - loss: 0.0251 - accuracy: 0.9930

********************************************************************************

Now training: define_large_model on dataset repeated 8x with batch size 256 on 2 gpu(s)
Number of devices: 2
Epoch 1/2
      1/Unknown - 6s 6s/step - loss: 2.3014 - accuracy: 0.1328WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.208290). Check your callbacks.


938/938 [==============================] - 162s 173ms/step - loss: 0.0812 - accuracy: 0.9742
Epoch 2/2
938/938 [==============================] - 59s 62ms/step - loss: 0.0089 - accuracy: 0.9971

********************************************************************************

Now training: define_small_model on dataset repeated 8x with batch size 512 on 1 gpu(s)
Epoch 1/2
938/938 [==============================] - 144s 153ms/step - loss: 0.1197 - accuracy: 0.9677
Epoch 2/2
938/938 [==============================] - 12s 13ms/step - loss: 0.0223 - accuracy: 0.9941

********************************************************************************

Now training: define_large_model on dataset repeated 8x with batch size 512 on 1 gpu(s)
Epoch 1/2
938/938 [==============================] - 208s 222ms/step - loss: 0.0804 - accuracy: 0.9742
Epoch 2/2
938/938 [==============================] - 100s 107ms/step - loss: 0.0087 - accuracy: 0.9971

*****************************************************

469/469 [==============================] - 140s 298ms/step - loss: 0.1861 - accuracy: 0.9507
Epoch 2/2
469/469 [==============================] - 8s 18ms/step - loss: 0.0365 - accuracy: 0.9899

********************************************************************************

Now training: define_large_model on dataset repeated 8x with batch size 512 on 2 gpu(s)
Number of devices: 2
Epoch 1/2
469/469 [==============================] - 154s 328ms/step - loss: 0.1224 - accuracy: 0.9617
Epoch 2/2
469/469 [==============================] - 55s 116ms/step - loss: 0.0129 - accuracy: 0.9959


In [10]:
! cat results.txt

{"n_gpus": 1, "second epoch time": 3.526135999999994, "batch_size": 128, "n_dataset_repeat": 1, "first epoch time": 35.357262}
{"n_gpus": 1, "second epoch time": 12.653536000000003, "batch_size": 128, "n_dataset_repeat": 1, "first epoch time": 46.01204200000001}
{"n_gpus": 2, "second epoch time": 4.091595000000012, "batch_size": 128, "n_dataset_repeat": 1, "first epoch time": 38.85839100000001}
{"n_gpus": 2, "second epoch time": 7.8085109999999815, "batch_size": 128, "n_dataset_repeat": 1, "first epoch time": 43.34130999999999}
{"n_gpus": 1, "second epoch time": 2.1493070000000216, "batch_size": 256, "n_dataset_repeat": 1, "first epoch time": 34.963386000000014}
{"n_gpus": 1, "second epoch time": 10.277569000000028, "batch_size": 256, "n_dataset_repeat": 1, "first epoch time": 42.76192099999997}
{"n_gpus": 2, "second epoch time": 2.3124480000000176, "batch_size": 256, "n_dataset_repeat": 1, "first epoch time": 36.45837400000005}
{"n_gpus": 2, "second epoch time": 4.762603999999953, "ba